In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
# Some format settings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Set the display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
# Function to clean individual year data with column name mapping and handling missing ALC_DRUG and Hispanic data before 2019
def clean_neiss_data(data):
    # Step 1: Standardize column names for Narrative and Sex
    # For 2022 and 2023, data used 'Narrative_1' instead of 'Narrative'
    # For 2023, data used 'Gender' instead of 'Sex'
    if 'Narrative_1' in data.columns:
        data.rename(columns={'Narrative_1': 'Narrative'}, inplace=True)
    if 'Gender' in data.columns:
        data.rename(columns={'Gender': 'Sex'}, inplace=True)
    
    # Step 2: Convert 'CPSC_Case_Number' to string
    data['CPSC_Case_Number'] = data['CPSC_Case_Number'].astype(str)
    
    # Step 3: Convert 'Treatment_Date' to datetime and extract year, month, and day
    # Add this step for better grouping
    data['Treatment_Date'] = pd.to_datetime(data['Treatment_Date'], errors='coerce')
    data['Year'] = data['Treatment_Date'].dt.year
    data['Month'] = data['Treatment_Date'].dt.month
    data['Day'] = data['Treatment_Date'].dt.day
    
    # Step 4: Fill missing values for numeric columns with a default number 0
    data['Other_Race'] = data['Other_Race'].fillna(0)
    data['Body_Part_2'] = data['Body_Part_2'].fillna(0)
    
    # Step 5: Fill missing values for string columns with a default string "None"
    data['Other_Diagnosis'] = data['Other_Diagnosis'].fillna('None')
    data['Diagnosis_2'] = data['Diagnosis_2'].fillna('None')
    data['Other_Diagnosis_2'] = data['Other_Diagnosis_2'].fillna('None')
    
    # Step 6: Fill Hispanic, Alcohol, and Drug with 0 (unknown) if Year < 2019
    data.loc[data['Year'] < 2019, 'Hispanic'] = data['Hispanic'].fillna(0)
    data.loc[data['Year'] < 2019, 'Alcohol'] = data['Alcohol'].fillna(0)
    data.loc[data['Year'] < 2019, 'Drug'] = data['Drug'].fillna(0)
    
    # Step 7: Clean the Narrative text
    data['Narrative'] = data['Narrative'].str.replace(r'[^a-zA-Z0-9*\s]', '', regex=True)
    
    return data

In [4]:
# Step 8: Consolidate data from multiple years
def consolidate_data(years):
    # Initialize an empty DataFrame to store all cleaned data
    consolidated_data = pd.DataFrame()
    
    # Iterate over each year and clean the corresponding dataset
    for year in years:
        file_path = f'NEISS_{year}.csv'
        data = pd.read_csv(file_path)
        
        # Clean the data
        cleaned_data = clean_neiss_data(data)
        
        # Append to consolidated data
        consolidated_data = pd.concat([consolidated_data, cleaned_data], ignore_index=True)
    
    return consolidated_data

In [5]:
# Step 9: List of years to process
years = list(range(2014, 2024))  # From 2014 to 2023

# Step 10: Clean and consolidate data
consolidated_data = consolidate_data(years)

# Step 11: Check for missing values in consolidated data
missing_values = consolidated_data.isnull().sum()
print("### Missing values in each column after consolidation ###\n", missing_values)

# Step 12: Save the cleaned and consolidated dataset
consolidated_data.to_csv('consolidated_cleaned_neiss_2014_2023.csv', index=False)

# Display the first few rows of the cleaned and consolidated data
consolidated_data.head()

### Missing values in each column after consolidation ###
 CPSC_Case_Number     0
Treatment_Date       0
Age                  0
Sex                  0
Race                 0
Other_Race           0
Hispanic             0
Body_Part            0
Diagnosis            0
Other_Diagnosis      0
Body_Part_2          0
Diagnosis_2          0
Other_Diagnosis_2    0
Disposition          0
Location             0
Fire_Involvement     0
Product_1            0
Product_2            0
Product_3            0
Alcohol              0
Drug                 0
Narrative            0
Stratum              0
PSU                  0
Weight               0
Year                 0
Month                0
Day                  0
dtype: int64


,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,Other_Diagnosis,Body_Part_2,Diagnosis_2,Other_Diagnosis_2,Disposition,Location,Fire_Involvement,Product_1,Product_2,Product_3,Alcohol,Drug,Narrative,Stratum,PSU,Weight,Year,Month,Day
0,140103999,2014-01-01,32,1,1,0,0.0,31,53,None,0.0,None,None,1,0,0,1211,0,0,0.0,0.0,32 YOM CO PAIN IN THE CHEST AFTER FALLING WHIL...,S,98,80.0213,2014,1,1
1,140104001,2014-01-01,18,1,1,0,0.0,82,51,None,0.0,None,None,1,1,2,910,0,0,0.0,0.0,18 YOM BURNED LT HAND WHILE POURING GASOLINE O...,S,98,80.0213,2014,1,1
2,140104003,2014-01-01,31,2,1,0,0.0,37,64,None,0.0,None,None,1,1,0,1807,0,0,0.0,0.0,31 YOF STATES SHE FELL THROUGH THE BATHROOM FL...,S,98,80.0213,2014,1,1
3,140104670,2014-01-01,53,2,1,0,0.0,82,59,None,0.0,None,None,1,1,0,478,0,0,0.0,0.0,53YOF WASHING DISHES AT HOME A GLASS BROKE AND...,S,29,80.0213,2014,1,1
4,140104672,2014-01-01,23,1,1,0,0.0,30,57,None,0.0,None,None,1,9,0,3286,0,0,0.0,0.0,23YOM RIDING AN ATV ROLLED OVER HURT SHOULDERD...,S,29,80.0213,2014,1,1
